# Importing Libraries

In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from collections import Counter

# Loading Data

In [17]:
df = pd.read_csv("results.csv")

# Data Cleaning

In [18]:
def count_unique_digits_with_leading_zeros(number):
    number_str = str(number).zfill(4)  
    return len(set(number_str)) 

def most_frequent_digit(number):
    number_str = str(number).zfill(4)
    digit_counts = {digit: number_str.count(digit) for digit in number_str}
    max_frequency = max(digit_counts.values())
    
    most_frequent = min([digit for digit, count in digit_counts.items() if count == max_frequency])
    
    return int(most_frequent)

# Apply functions
df['distinct_values'] = df['Number'].apply(count_unique_digits_with_leading_zeros)
df['most_frequent_digit'] = df['Number'].apply(most_frequent_digit)

df.loc[df['distinct_values'] == 4, 'most_frequent_digit'] = df['Number'].astype(str).str.zfill(4).str[0].astype(int)

df['Number'] = df['Number'].astype(str).str.zfill(4)

# Extract individual digits as target variables
df['Digit_1'] = df['Number'].str[0].astype(int)
df['Digit_2'] = df['Number'].str[1].astype(int)
df['Digit_3'] = df['Number'].str[2].astype(int)
df['Digit_4'] = df['Number'].str[3].astype(int)

df


,Draw Number,Number,Prize,Date,distinct_values,most_frequent_digit,Digit_1,Digit_2,Digit_3,Digit_4
0,5304,4111,1,23/3/2025,2,1,4,1,1,1
1,5304,4035,2,23/3/2025,4,4,4,0,3,5
2,5304,4379,3,23/3/2025,4,4,4,3,7,9
3,5304,0188,4,23/3/2025,3,8,0,1,8,8
4,5304,2497,4,23/3/2025,4,2,2,4,9,7
...,...,...,...,...,...,...,...,...,...,...
409,5287,8858,5,12/2/2025,2,8,8,8,5,8
410,5287,8874,5,12/2/2025,3,8,8,8,7,4
411,5287,9199,5,12/2/2025,2,9,9,1,9,9
412,5287,9435,5,12/2/2025,4,9,9,4,3,5


In [19]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', dayfirst=True)

# Extract Year, Month, and Day
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Extract Weekday as a number (Monday = 1, Sunday = 7)
df['Weekday'] = df['Date'].dt.weekday + 1  # Adding 1 to make Monday = 1

df

,Draw Number,Number,Prize,Date,distinct_values,most_frequent_digit,Digit_1,Digit_2,Digit_3,Digit_4,Year,Month,Day,Weekday
0,5304,4111,1,2025-03-23,2,1,4,1,1,1,2025,3,23,7
1,5304,4035,2,2025-03-23,4,4,4,0,3,5,2025,3,23,7
2,5304,4379,3,2025-03-23,4,4,4,3,7,9,2025,3,23,7
3,5304,0188,4,2025-03-23,3,8,0,1,8,8,2025,3,23,7
4,5304,2497,4,2025-03-23,4,2,2,4,9,7,2025,3,23,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,5287,8858,5,2025-02-12,2,8,8,8,5,8,2025,2,12,3
410,5287,8874,5,2025-02-12,3,8,8,8,7,4,2025,2,12,3
411,5287,9199,5,2025-02-12,2,9,9,1,9,9,2025,2,12,3
412,5287,9435,5,2025-02-12,4,9,9,4,3,5,2025,2,12,3


In [20]:
selected_feature = ['Draw Number', 'distinct_values', 'most_frequent_digit', 'Year', 'Month', 'Day', 'Weekday', 'Prize']
target_cols = ['Digit_1', 'Digit_2', 'Digit_3', 'Digit_4']

# Data Splitting

# Split into train, validate and test

In [21]:
train_validate_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

X_test = test_df[selected_feature]
y_test = test_df[target_cols]

print('X test shape:', X_test.shape)
print('y test shape:', y_test.shape)

X test shape: (63, 8)
y test shape: (63, 4)


In [22]:
X = train_validate_df[selected_feature]  # Features
y = train_validate_df[target_cols]  # Target digits

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=42)

print('X train shape:', X_train.shape)
print('y train shape:', y_train.shape)

print('X validate shape:', X_validate.shape)
print('y validate shape:', y_validate.shape)

X train shape: (280, 8)
y train shape: (280, 4)
X validate shape: (71, 8)
y validate shape: (71, 4)


# Training the model

## Extra Trees

In [23]:
et_model_config = {
    'n_estimators': 300,
    'max_depth': 25,
    'criterion': 'gini',
    'random_state': 42,
    'bootstrap': True
}

et_model = ExtraTreesClassifier(**et_model_config)
et_multi_model = MultiOutputClassifier(et_model)
et_multi_model.fit(X_train, y_train)

# Get probability predictions for each target digit
y_pred_probability_et = et_multi_model.predict_proba(X_validate)

# Prepare DataFrame to store results
results = []

# Process each row in validation set
for idx in range(len(X_validate)):
    row_results = {}

    # Store actual values and get top 1 prediction for each digit
    pred_1_score = 0
    pred_2_score = 0

    for digit in range(4):  # Iterate over all 4 digit columns
        actual_value = y_validate.iloc[idx, digit]
        row_results[f'Digit_{digit+1}'] = actual_value  # Actual value

        # Get top 2 predictions
        digit_proba = y_pred_probability_et[digit][idx]
        top_2_indices = np.argsort(digit_proba)[-2:][::-1]  # Top 2 predicted digits

        # Store predictions
        row_results[f'Pred_1_Digit_{digit+1}'] = int(top_2_indices[0])
        row_results[f'Pred_2_Digit_{digit+1}'] = int(top_2_indices[1])

        # Assign scores (1 if correct, 0 if incorrect)
        row_results[f'Pred_1_Digit_{digit+1}_Score'] = int(actual_value == top_2_indices[0])
        row_results[f'Pred_2_Digit_{digit+1}_Score'] = int(actual_value == top_2_indices[1])

        # Update total score counters
        pred_1_score += row_results[f'Pred_1_Digit_{digit+1}_Score']
        pred_2_score += row_results[f'Pred_2_Digit_{digit+1}_Score']

    # Compute final scores as percentage (correct digits / 4)
    row_results["pred_1_match_score"] = pred_1_score / 4
    row_results["pred_2_match_score"] = pred_2_score / 4

    results.append(row_results)

# Convert to DataFrame
et_df = pd.DataFrame(results)

# Create full number representation
et_df["number"] = et_df[[f"Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
et_df["pred_1"] = et_df[[f"Pred_1_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
et_df["pred_2"] = et_df[[f"Pred_2_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)

def digit_frequency_match_percentage(num1, num2):
    """Returns the percentage of digit matches between two numbers based on frequency."""
    counter1 = Counter(str(num1).zfill(4))  # Ensure 4-digit format
    counter2 = Counter(str(num2).zfill(4))
    
    # Count the total number of matching digits based on frequency
    matching_digits = sum(min(counter1[d], counter2[d]) for d in counter1)
    
    return matching_digits / 4  # Normalize to get percentage

# Apply function to compute match percentage
et_df["pred_1_sim_perc"] = et_df.apply(lambda row: digit_frequency_match_percentage(row["number"], row["pred_1"]), axis=1)
et_df["pred_2_sim_perc"] = et_df.apply(lambda row: digit_frequency_match_percentage(row["number"], row["pred_2"]), axis=1)

# Keep only relevant columns
et_df = et_df[["number", "pred_1", "pred_1_match_score", "pred_1_sim_perc",
               "pred_2", "pred_2_match_score", "pred_2_sim_perc"]]

et_df

,number,pred_1,pred_1_match_score,pred_1_sim_perc,pred_2,pred_2_match_score,pred_2_sim_perc
0,3162,3610,0.25,0.75,4359,0.00,0.25
1,3027,4619,0.00,0.00,3870,0.25,0.75
2,1521,2523,0.50,0.50,0291,0.25,0.50
3,3405,3910,0.25,0.50,2646,0.00,0.25
4,8718,7886,0.00,0.75,9534,0.00,0.00
...,...,...,...,...,...,...,...
66,0505,0523,0.50,0.50,2240,0.00,0.25
67,3144,4228,0.00,0.25,5044,0.50,0.50
68,5214,5012,0.50,0.75,6158,0.00,0.50
69,5097,4107,0.25,0.50,6292,0.25,0.25


## Random Forest

In [24]:
rf_model = RandomForestClassifier(n_estimators=300, random_state=1337)
rf_multi_model = MultiOutputClassifier(rf_model)

rf_multi_model.fit(X_train, y_train)

# Make Predictions
y_pred_probability_rf = rf_multi_model.predict_proba(X_validate)

# Prepare DataFrame to store results
results = []

# Process each row in validation set
for idx in range(len(X_validate)):
    row_results = {}

    # Store actual values and get top 1 prediction for each digit
    pred_1_score = 0
    pred_2_score = 0

    for digit in range(4):  # Iterate over all 4 digit columns
        actual_value = y_validate.iloc[idx, digit]
        row_results[f'Digit_{digit+1}'] = actual_value  # Actual value

        # Get top 2 predictions
        digit_proba = y_pred_probability_rf[digit][idx]
        top_2_indices = np.argsort(digit_proba)[-2:][::-1]  # Top 2 predicted digits

        # Store predictions
        row_results[f'Pred_1_Digit_{digit+1}'] = int(top_2_indices[0])
        row_results[f'Pred_2_Digit_{digit+1}'] = int(top_2_indices[1])

        # Assign scores (1 if correct, 0 if incorrect)
        row_results[f'Pred_1_Digit_{digit+1}_Score'] = int(actual_value == top_2_indices[0])
        row_results[f'Pred_2_Digit_{digit+1}_Score'] = int(actual_value == top_2_indices[1])

        # Update total score counters
        pred_1_score += row_results[f'Pred_1_Digit_{digit+1}_Score']
        pred_2_score += row_results[f'Pred_2_Digit_{digit+1}_Score']

    # Compute final scores as percentage (correct digits / 4)
    row_results["pred_1_match_score"] = pred_1_score / 4
    row_results["pred_2_match_score"] = pred_2_score / 4

    results.append(row_results)

# Convert to DataFrame
rf_df = pd.DataFrame(results)

# Create full number representation
rf_df["number"] = rf_df[[f"Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
rf_df["pred_1"] = rf_df[[f"Pred_1_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
rf_df["pred_2"] = rf_df[[f"Pred_2_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)

def digit_frequency_match_percentage(num1, num2):
    """Returns the percentage of digit matches between two numbers based on frequency."""
    counter1 = Counter(str(num1).zfill(4))  # Ensure 4-digit format
    counter2 = Counter(str(num2).zfill(4))
    
    # Count the total number of matching digits based on frequency
    matching_digits = sum(min(counter1[d], counter2[d]) for d in counter1)
    
    return matching_digits / 4  # Normalize to get percentage

# Apply function to compute match percentage
rf_df["pred_1_sim_perc"] = rf_df.apply(lambda row: digit_frequency_match_percentage(row["number"], row["pred_1"]), axis=1)
rf_df["pred_2_sim_perc"] = rf_df.apply(lambda row: digit_frequency_match_percentage(row["number"], row["pred_2"]), axis=1)

# Keep only relevant columns
rf_df = rf_df[["number", "pred_1", "pred_1_match_score", "pred_1_sim_perc",
               "pred_2", "pred_2_match_score", "pred_2_sim_perc"]]

rf_df

,number,pred_1,pred_1_match_score,pred_1_sim_perc,pred_2,pred_2_match_score,pred_2_sim_perc
0,3162,3359,0.25,0.25,2886,0.00,0.50
1,3027,2870,0.00,0.75,3619,0.25,0.25
2,1521,2291,0.25,0.50,0523,0.50,0.50
3,3405,3910,0.25,0.50,2646,0.00,0.25
4,8718,7886,0.00,0.75,1394,0.00,0.25
...,...,...,...,...,...,...,...
66,0505,0523,0.50,0.50,2248,0.00,0.00
67,3144,4224,0.25,0.50,5548,0.25,0.25
68,5214,5019,0.50,0.50,4592,0.00,0.75
69,5097,4107,0.25,0.50,5292,0.50,0.50


## XGBoost

In [25]:
xgb_model_config = {
    'objective': "multi:softmax",  # Multi-class classification
    "num_class": 10,  # Number of target classes
    'n_estimators': 100,  # Number of trees (boosting rounds)
    'learning_rate': 0.1,  # Learning rate (eta)
    'max_depth': 5,  # Max depth of trees
    'min_child_weight': 2,  # Minimum sum of instance weight in child
    "eval_metric": "mlogloss",  # Multi-class log loss
    'random_state': 42
}

xgb_model = xgb.XGBClassifier(**{**xgb_model_config})
xgb_multi = MultiOutputClassifier(xgb_model)

xgb_multi.fit(X_train, y_train)

# Make Predictions
y_pred_probability_xgb = xgb_multi.predict_proba(X_validate)

# Prepare DataFrame to store results
results = []

# Process each row in validation set
for idx in range(len(X_validate)):
    row_results = {}

    # Store actual values and get top 1 prediction for each digit
    pred_1_score = 0
    pred_2_score = 0

    for digit in range(4):  # Iterate over all 4 digit columns
        actual_value = y_validate.iloc[idx, digit]
        row_results[f'Digit_{digit+1}'] = actual_value  # Actual value

        # Get top 2 predictions
        digit_proba = y_pred_probability_xgb[digit][idx]
        top_2_indices = np.argsort(digit_proba)[-2:][::-1]  # Top 2 predicted digits

        # Store predictions
        row_results[f'Pred_1_Digit_{digit+1}'] = int(top_2_indices[0])
        row_results[f'Pred_2_Digit_{digit+1}'] = int(top_2_indices[1])

        # Assign scores (1 if correct, 0 if incorrect)
        row_results[f'Pred_1_Digit_{digit+1}_Score'] = int(actual_value == top_2_indices[0])
        row_results[f'Pred_2_Digit_{digit+1}_Score'] = int(actual_value == top_2_indices[1])

        # Update total score counters
        pred_1_score += row_results[f'Pred_1_Digit_{digit+1}_Score']
        pred_2_score += row_results[f'Pred_2_Digit_{digit+1}_Score']

    # Compute final scores as percentage (correct digits / 4)
    row_results["pred_1_match_score"] = pred_1_score / 4
    row_results["pred_2_match_score"] = pred_2_score / 4

    results.append(row_results)

# Convert to DataFrame
xgb_df = pd.DataFrame(results)

# Create full number representation
xgb_df["number"] = xgb_df[[f"Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
xgb_df["pred_1"] = xgb_df[[f"Pred_1_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
xgb_df["pred_2"] = xgb_df[[f"Pred_2_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)

def digit_frequency_match_percentage(num1, num2):
    """Returns the percentage of digit matches between two numbers based on frequency."""
    counter1 = Counter(str(num1).zfill(4))  # Ensure 4-digit format
    counter2 = Counter(str(num2).zfill(4))
    
    # Count the total number of matching digits based on frequency
    matching_digits = sum(min(counter1[d], counter2[d]) for d in counter1)
    
    return matching_digits / 4  # Normalize to get percentage

# Apply function to compute match percentage
xgb_df["pred_1_sim_perc"] = xgb_df.apply(lambda row: digit_frequency_match_percentage(row["number"], row["pred_1"]), axis=1)
xgb_df["pred_2_sim_perc"] = xgb_df.apply(lambda row: digit_frequency_match_percentage(row["number"], row["pred_2"]), axis=1)

# Keep only relevant columns
xgb_df = xgb_df[["number", "pred_1", "pred_1_match_score", "pred_1_sim_perc",
                 "pred_2", "pred_2_match_score", "pred_2_sim_perc"]]

xgb_df

,number,pred_1,pred_1_match_score,pred_1_sim_perc,pred_2,pred_2_match_score,pred_2_sim_perc
0,3162,3659,0.25,0.50,2082,0.25,0.25
1,3027,3619,0.25,0.25,2057,0.50,0.75
2,1521,1191,0.50,0.50,2403,0.00,0.25
3,3405,3616,0.25,0.25,4145,0.25,0.50
4,8718,7896,0.00,0.50,9487,0.00,0.50
...,...,...,...,...,...,...,...
66,0505,0400,0.50,0.50,1793,0.00,0.00
67,3144,4294,0.25,0.50,5048,0.25,0.25
68,5214,5099,0.25,0.25,6552,0.00,0.50
69,5097,5107,0.50,0.75,3290,0.25,0.50


# Testing model

In [26]:
# Make Predictions on Test Data
y_test_pred = et_multi_model.predict(X_test)

# Convert Predictions to String Format
predicted_numbers = [''.join(map(str, digits)) for digits in y_test_pred]

# Get Actual Numbers from Test Data
actual_numbers = [''.join(map(str, row)) for row in y_test.values]

# Compare Predictions with Actual Values
correct_predictions = sum([1 if pred == actual else 0 for pred, actual in zip(predicted_numbers, actual_numbers)])
total_predictions = len(test_df)

# Calculate Accuracy
accuracy = correct_predictions / total_predictions * 100

# Display Results
print(f"Predicted Numbers: {predicted_numbers}")
print(f"Actual Numbers:    {actual_numbers}")
print(f"Correct Predictions: {correct_predictions} out of {total_predictions}")
print(f"Lottery Number Accuracy: {accuracy:.2f}%")

Predicted Numbers: ['8010', '4539', '7233', '8025', '0243', '1849', '9233', '0171', '6386', '1879', '0870', '3610', '2337', '5016', '6057', '9353', '7016', '6065', '7984', '0956', '3312', '5792', '7356', '9797', '9675', '9025', '3194', '6548', '5847', '1202', '7162', '5337', '9326', '8978', '7162', '5887', '2607', '9199', '3645', '4984', '5131', '7796', '9371', '6710', '7649', '5545', '7841', '3377', '9649', '1586', '8425', '7610', '4984', '9649', '2473', '1379', '6684', '6673', '6548', '0313', '8025', '2672', '4587']
Actual Numbers:    ['0240', '4297', '6433', '8025', '6464', '1054', '5339', '0032', '8299', '0187', '0156', '3801', '2124', '6913', '8123', '8019', '7590', '3855', '7178', '0876', '3368', '5310', '7097', '9980', '9306', '9478', '3571', '6906', '4260', '8700', '7053', '0313', '9014', '6666', '7940', '1399', '2415', '8858', '5468', '4525', '1197', '9848', '4111', '6285', '6372', '5955', '4797', '8680', '9873', '1940', '8047', '6784', '6634', '9380', '2196', '1186', '6056', 

In [27]:
# Make Predictions on Test Data
y_test_pred = et_multi_model.predict_proba(X_test)

# Prepare DataFrame to store results
results = []

# Process each row in validation set
for idx in range(len(X_test)):
    row_results = {}

    # Store actual values and get top 2 predictions for each digit
    for digit in range(4):  # Iterate over all 4 digit columns
        actual_value = y_test.iloc[idx, digit]
        row_results[f'Digit_{digit+1}'] = actual_value  # Actual value

        # Get top 2 predictions and their probabilities for the current digit
        digit_proba = y_test_pred[digit][idx]  # Probabilities for this digit
        top_2_indices = np.argsort(digit_proba)[-2:][::-1]  # Get top 2 class indices
        top_2_probs = np.sort(digit_proba)[-2:][::-1]  # Get top 2 probabilities

        # Store predictions in separate columns
        row_results[f'Pred_1_Digit_{digit+1}'] = int(top_2_indices[0])
        row_results[f'Prob_1_Digit_{digit+1}'] = float(top_2_probs[0])
        row_results[f'Pred_2_Digit_{digit+1}'] = int(top_2_indices[1])
        row_results[f'Prob_2_Digit_{digit+1}'] = float(top_2_probs[1])

    results.append(row_results)

# Convert to DataFrame
results_df = pd.DataFrame(results)

results_df

,Digit_1,Pred_1_Digit_1,Prob_1_Digit_1,Pred_2_Digit_1,Prob_2_Digit_1,Digit_2,Pred_1_Digit_2,Prob_1_Digit_2,Pred_2_Digit_2,Prob_2_Digit_2,Digit_3,Pred_1_Digit_3,Prob_1_Digit_3,Pred_2_Digit_3,Prob_2_Digit_3,Digit_4,Pred_1_Digit_4,Prob_1_Digit_4,Pred_2_Digit_4,Prob_2_Digit_4
0,0,8,0.646667,0,0.186667,2,0,0.711667,3,0.103333,4,1,0.653333,3,0.150000,0,0,0.783333,5,0.056667
1,4,4,0.750000,3,0.113333,2,5,0.750000,9,0.163333,9,3,0.640000,8,0.160000,7,9,0.660000,5,0.170000
2,6,7,0.353333,0,0.213333,4,2,0.496667,3,0.200000,3,3,0.536667,8,0.136667,3,3,0.403275,6,0.295058
3,8,8,0.626667,9,0.166667,0,0,0.610000,4,0.103333,2,2,0.620000,7,0.121352,5,5,0.746667,4,0.076667
4,6,0,0.203333,9,0.196667,4,2,0.246667,8,0.210000,6,4,0.240000,6,0.216667,4,3,0.169259,9,0.163333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,4,6,0.217722,4,0.204444,7,5,0.226667,3,0.214611,8,4,0.254833,9,0.163333,4,8,0.249556,6,0.229889
59,3,0,0.606667,3,0.256667,4,3,0.646667,4,0.130000,7,1,0.610000,4,0.141667,3,3,0.616667,6,0.160000
60,8,8,0.626667,9,0.166667,1,0,0.610000,4,0.103333,5,2,0.620000,7,0.121352,9,5,0.746667,4,0.076667
61,2,2,0.533333,3,0.130000,5,6,0.563333,3,0.116667,1,7,0.623333,1,0.110000,2,2,0.506667,3,0.140000


In [28]:
results_df["Number"] = results_df[[f"Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)

results_df["Predicted_Number_1"] = results_df[[f"Pred_1_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
results_df["Predicted_Number_1_Prob"] = results_df[[f"Prob_1_Digit_{i}" for i in range(1, 5)]].mean(axis=1)

results_df["Predicted_Number_2"] = results_df[[f"Pred_2_Digit_{i}" for i in range(1, 5)]].astype(str).agg("".join, axis=1)
results_df["Predicted_Number_2_Prob"] = results_df[[f"Prob_2_Digit_{i}" for i in range(1, 5)]].mean(axis=1)

def unordered_digit_match(num1, num2):
    return int(Counter(str(num1)) == Counter(str(num2)))  # Compare digit counts

results_df["Pred_1_Score"] = results_df.apply(lambda row: unordered_digit_match(row["Number"], row["Predicted_Number_1"]), axis=1)

results_df["Pred_2_Score"] = results_df.apply(lambda row: unordered_digit_match(row["Number"], row["Predicted_Number_2"]), axis=1)

final_df = results_df[["Number", "Predicted_Number_1", "Predicted_Number_1_Prob", "Pred_1_Score",
                       "Predicted_Number_2", "Predicted_Number_2_Prob", "Pred_2_Score"]]

final_df

,Number,Predicted_Number_1,Predicted_Number_1_Prob,Pred_1_Score,Predicted_Number_2,Predicted_Number_2_Prob,Pred_2_Score
0,0240,8010,0.698750,0,0335,0.124167,0
1,4297,4539,0.700000,0,3985,0.151667,0
2,6433,7233,0.447485,0,0386,0.211265,0
3,8025,8025,0.650833,1,9474,0.117005,0
4,6464,0243,0.214815,0,9869,0.196667,0
...,...,...,...,...,...,...,...
58,4784,6548,0.237194,0,4396,0.203069,0
59,3473,0313,0.620000,0,3446,0.172083,0
60,8159,8025,0.650833,0,9474,0.117005,0
61,2512,2672,0.556667,0,3313,0.124167,0
